In [30]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
import sqlite3
import shutil

In [31]:
# --- Al final, enviar correo ---
def enviar_correo(asunto, Texto_Enciado, destinatario):
    remitente = "pcaliman@gmail.com"           # Cambiar por tu email
    clave = "glhz qhos bbww sgyc"              # Tu contraseña
    servidor_smtp = "smtp.gmail.com"
    puerto = 587

    try:
        # Crear el mensaje
        mensaje = MIMEMultipart()
        mensaje["From"] = remitente
        mensaje["To"] = destinatario
        mensaje["Subject"] = asunto
        mensaje.attach(MIMEText(Texto_Enciado, "html"))

        # Conectarse al servidor SMTP
        servidor = smtplib.SMTP(servidor_smtp, puerto)
        servidor.starttls()
        servidor.login(remitente, clave)
        servidor.send_message(mensaje)
        servidor.quit()

        print("✅ Correo enviado correctamente.")
    except Exception as e:
        print("❌ Error al enviar correo:", e)

Categg = ['Grab And Go', 'Cafeteria', 'Snacks',  'Almacen', 'Non Food & In/Out', 'Liquidos - Cooler', 'Fast Food']


def Borradoo(archivo): 
    import os

    #archivo = "./archivo.txt"

    try:
        os.remove(archivo)
        print("Archivo eliminado exitosamente.")
    except FileNotFoundError:
        print("El archivo no existe.")
    except PermissionError:
        print("No tienes permisos para eliminar este archivo.")

def Archivo2Work():
    directorio =  os.getcwd() + '\Inventario'
    archivos_excel = []
    countador = 0
    for archivo in os.listdir(directorio):
        if archivo.endswith(".xlsx"):
            archivos_excel.append(archivo)
        ArchivoPAraWork =archivos_excel[0]
        countador += 1
    if (countador != 0):
        return ArchivoPAraWork, directorio, True
    return "Nada", directorio, False


def update_tienda_bodega(df, db_path="inventaria.db"):
    with sqlite3.connect(db_path, timeout=10) as conn:
        cursor = conn.cursor()
        for _, fila in df.iterrows():
            cursor.execute("""
                UPDATE omega
                SET tienda = ?, 
                    bodega = ?
                WHERE sku = ?
            """, (fila['Tienda'], fila['Bodega'], fila['SKU']))
        conn.commit()


def recalcular_faltante(db_path="inventaria.db"):

    with sqlite3.connect(db_path, timeout=10) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE omega
            SET faltante = maximos - tienda
        """)
        conn.commit()


def copy_bodega_to_anterior(db_path="inventaria.db"):

    with sqlite3.connect(db_path, timeout=10) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE omega
            SET anterior = bodega
        """)
        conn.commit()

def leer_excel_a_pp(directorio, archivo):
    RutaFile = os.path.join(directorio, archivo)
    pp = pd.read_excel(RutaFile, header=None)
    pp.columns = pp.iloc[4]
    pp = pp[5:].reset_index(drop=True)
    pp = pp.loc[:, ~pp.columns.isna()]
    pp['SKU'] = pp['SKU'].fillna(0).astype(int)
    pp = pp[pp['SKU'] != 0]
    pp.columns = ['Producto', 'Categoria', 'SKU', 'Tienda', 'Bodega', 'Total', 'Unidad']
    pp['Tienda'] = pp['Tienda'].fillna(0).astype(int)
    pp['Bodega'] = pp['Bodega'].fillna(0).astype(int)
    return pp

In [32]:
def mover_archivo(directorioop, FilePAraWork):
    ruta = directorioop + FilePAraWork
    directorio = os.path.dirname(ruta)          # -> C:\Inventario\Subcarpeta
    directorio_padre = os.path.dirname(directorio)  # -> C:\Inventario
    # Nueva ruta en el directorio padre
    carpeta_old = os.path.join(directorio_padre, "old")
    os.makedirs(carpeta_old, exist_ok=True)  # Crear si no existe
    ruta_destino = os.path.join(carpeta_old, os.path.basename(ruta))
     # Mover el archivo
    shutil.move(ruta, ruta_destino)


    carpeta_output = os.path.join(directorio_padre, "output")
    os.makedirs(carpeta_output, exist_ok=True)  # Crear si no existe

    NombSalida = os.path.join(carpeta_output, 'Salida_' + FilePAraWork)  
    data.to_excel(NombSalida, index=True)

In [33]:
def generar_y_enviar_correo(ASuntoDato, data, Categg):
    asunto = f"Movimientos: {ASuntoDato}"
    Texto_Enviado = """
    <p>Buen día,</p>
    <p><strong>Productos con faltante:</strong></p>
    """

    for iiCat in Categg:
        Texto_Enviado += f"<p style='color:green; font-weight:bold;'>- {iiCat} -</p>"
        for index, row in data.iterrows():
            tienda = row['Tienda']
            bodega = row['Bodega']
            producto = row['Producto']
            faltante = row['Faltante']
            categoriaa = row['Categoria']
            MMaximos = row['Maximos']
            anteriorr = row['Anterior']
            if (faltante > 0) and (iiCat == categoriaa):
                # Color según bodega
                if bodega >=  faltante:
                    if anteriorr == bodega:
                        color="red"
                    else:
                        color = "blue"
                else:
                    color = "orange"

                Texto_Enviado += (
                    f"<span style='color:{color};'>"
                    f"- {producto}: SKU: {index} | "
                    f"Und. Sugeridas Tienda: {MMaximos} | "
                    f"Transferir: {faltante} unidades | "
                    f"Tienda: {tienda} | Bodega: {bodega}"
                    f"</span><br>"
                )

    Texto_Enviado += "<p><em>Este es un correo generado automáticamente.</em></p>"

    destinatarioo = "pcaliman@hotmail.com"
    ##destinatarioo = "pcaliman@hotmail.com, 20110@escopec.cl"
    #destinatarioo = "pcaliman@hotmail.com, 20110@escopec.cl, brayan@gfv.cl, luis@gfv.cl"
    enviar_correo(asunto, Texto_Enviado, destinatarioo)

In [34]:
FilePAraWork,directorioop, estadoos = Archivo2Work()
if estadoos:
    directorioop = directorioop + '\\'
    ASuntoDato=FilePAraWork
    pp = leer_excel_a_pp(directorioop, FilePAraWork)
    update_tienda_bodega(pp, "inventaria.db")
    recalcular_faltante("inventaria.db")
    conn = sqlite3.connect("inventaria.db")
    cursor = conn.cursor()
    data = pd.read_sql_query("SELECT * FROM omega", conn)
    data.columns = ['SKU','Producto', 'Categoria', 'Maximos', 'Tienda', 'Bodega', 'Total', 'Faltante', 'Anterior', 'Unidad']
    data.set_index('SKU', inplace=True)
    data.sort_values(by="Producto", inplace=True)
    generar_y_enviar_correo(ASuntoDato, data, Categg)
    conn.close()
    copy_bodega_to_anterior("inventaria.db")
    mover_archivo(directorioop, FilePAraWork)
else:
    print('Nada de nada')


C:\Users\Pedro Caliman\AppData\Local\Temp\ipykernel_5388\2867159796.py:100: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pp['Tienda'] = pp['Tienda'].fillna(0).astype(int)
C:\Users\Pedro Caliman\AppData\Local\Temp\ipykernel_5388\2867159796.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pp['Bodega'] = pp['Bodega'].fillna(0).astype(int)


✅ Correo enviado correctamente.


In [35]:
import sqlite3
import pandas as pd

def sku_not_in_db(df, db_path="inventaria.db"):
    """
    Devuelve los SKU presentes en el DataFrame df pero que NO existen en la tabla omega.
    df debe tener 'SKU' como columna (aunque sea el índice).
    """
    with sqlite3.connect(db_path, timeout=10) as conn:
        # Cargo los SKU de la tabla omega como Serie de pandas
        db_skus = pd.read_sql_query("SELECT sku FROM omega", conn)['sku']
        # Normalizo a string por seguridad
        db_skus = db_skus.astype(str).str.strip()

    # Normalizo los SKU del DataFrame
    df_skus = df['SKU'].astype(str).str.strip()

    # Filtro los que NO están en la base
    faltantes = df.loc[~df_skus.isin(db_skus)]

    return faltantes


In [36]:
import sqlite3

def insertar_nuevos(df_faltantes, db_path="inventaria.db"):
    """
    Inserta en la tabla 'nuevos' los registros que faltan en omega,
    manteniendo el SKU que viene del DataFrame.
    """
    with sqlite3.connect(db_path, timeout=10) as conn:
        cursor = conn.cursor()
        
        for _, fila in df_faltantes.iterrows():
            cursor.execute("""
                INSERT OR REPLACE INTO nuevos (sku, producto, categoria, tienda, bodega, total, unidad)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                int(fila['SKU']),        # mantener SKU numérico
                fila['Producto'],
                fila['Categoria'],
                fila['Tienda'],
                fila['Bodega'],
                fila['Total'],
                fila['Unidad']
            ))
        
        conn.commit()


In [37]:
faltantes = sku_not_in_db(pp)
insertar_nuevos(faltantes)



In [40]:
print(directorioop)
print(FilePAraWork)

c:\Proyecto_COPEC\Excel_w_Python_Project_Existencias\Inventario\
Listado_existencias_20250925.xlsx


In [58]:

directorio_raiz = os.path.dirname(directorioop) 
directorio_raiz = os.path.dirname(directorio_raiz) 

carpeta_old = os.path.join(directorio_raiz, "faltante")
os.makedirs(carpeta_old, exist_ok=True)  # Crear si no existe
NombSalida = os.path.join(carpeta_old, 'Nuevos_SKU_Sin_Maximos_Definidos.xlsx')  
print(directorio_raiz)
print(NombSalida)


c:\Proyecto_COPEC\Excel_w_Python_Project_Existencias
c:\Proyecto_COPEC\Excel_w_Python_Project_Existencias\faltante\Nuevos_SKU_Sin_Maximos_Definidos.xlsx


In [48]:
faltantes.head()

,Producto,Categoria,SKU,Tienda,Bodega,Total,Unidad
20,Muffin Relleno Manjar Refrigerado,Cafeteria,32231,-1,0,-1,unidad
21,Roulette Cheesecake,Cafeteria,32237,-1,0,-1,unidad
37,Panuelo Elite Aloe Vera,Almacen,130609,9,0,9,unidad
39,Kinder Bueno T2 43g,Snacks,130919,8,6,14,unidad
45,Duracell AAAx2,Non Food & In/Out,131046,0,60,60,unidad


In [52]:
print(NombSalida)

c:\Proyecto_COPEC\Excel_w_Python_Project_Existencias\Inventario\faltante\Nuevos_SKU_Sin_Maximos_Definidos.xlsx


In [59]:
faltantes.to_excel(NombSalida, index=True)